In [21]:
# Cell 1: Data Loading & Feature Engineering (FIXED)
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler

# 1. Load Data
df = pd.read_csv('../data/processed/sentiment_data.csv')

# 2. Convert Ratings & Sentiment to Numeric
score_cols = ['rating_1', 'rating_2', 'rating_3', 'rating_4', 'rating_5', 'rating_6']
for col in score_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')

df['is_pos'] = (df['bert_label'] == 'Positive').astype(int)
df['is_neg'] = (df['bert_label'] == 'Negative').astype(int)
df['is_neu'] = (df['bert_label'] == 'Neutral').astype(int)

# --- Handling Missing Columns ---
# 1. Department Check
if 'department' not in df.columns:
    df['department'] = 'General'

# 2. Course Name Check
# We map your CSV column 'course_name' to our internal variable 'lesson_name'
if 'course_name' in df.columns:
    df['lesson_name'] = df['course_name']
else:
    df['lesson_name'] = 'General'

# Ensure text columns are strings
df['department'] = df['department'].fillna('General').astype(str)
df['lesson_name'] = df['lesson_name'].fillna('General').astype(str)

if 'clean_comment_text' in df.columns:
    df['clean_comment_text'] = df['clean_comment_text'].astype(str)

# --- Advanced Keyword Detection ---
def check_keywords(text, keywords):
    if not isinstance(text, str): return 0
    return 1 if any(k in text for k in keywords) else 0

# Detect "Project"
df['has_project'] = df['clean_comment_text'].apply(lambda x: check_keywords(x, ['پروژه']))

# Detect "Homework"
df['has_homework'] = df['clean_comment_text'].apply(lambda x: check_keywords(x, ['تمرین', 'تکلیف', 'هومورک']))

# Detect "Attendance"
df['has_attendance'] = df['clean_comment_text'].apply(lambda x: check_keywords(x, ['حضور', 'غیبت', 'لیست']))
# ---------------------------------------

# 3. Create Professor Profiles
agg_dict = {
    'rating_1': 'mean',
    'rating_2': 'mean',
    'rating_3': 'mean',
    'rating_4': 'mean',
    'rating_5': 'mean',
    'rating_6': 'mean',
    'is_pos': 'mean',
    'is_neg': 'mean',
    'is_neu': 'mean',
    'has_project': 'mean',
    'has_homework': 'mean',
    'has_attendance': 'mean',
    'id': 'count',
    # LOGIC: Take the most frequent department
    'department': lambda x: x.mode()[0] if not x.mode().empty else 'General',
    # LOGIC: Join all unique course names
    'lesson_name': lambda x: ' | '.join(sorted(x.unique().astype(str)))
}

prof_profile = df.groupby('professor_name').agg(agg_dict).rename(columns={'id': 'comment_count'})

# 4. Filter (Keep profs with 5+ comments)
prof_profile = prof_profile[prof_profile['comment_count'] >= 5]

print(f"Total Professors: {len(prof_profile)}")
print("Columns:", prof_profile.columns.tolist())
display(prof_profile.head(3))

Total Professors: 293
Columns: ['rating_1', 'rating_2', 'rating_3', 'rating_4', 'rating_5', 'rating_6', 'is_pos', 'is_neg', 'is_neu', 'has_project', 'has_homework', 'has_attendance', 'comment_count', 'department', 'lesson_name']


,rating_1,rating_2,rating_3,rating_4,rating_5,rating_6,is_pos,is_neg,is_neu,has_project,has_homework,has_attendance,comment_count,department,lesson_name
professor_name,,,,,,,,,,,,,,,
ابراهیم اردشیر لاریجانی,6.250000,6.375000,4.875000,7.250000,6.625000,7.25,0.125000,0.750000,0.125,0.25,0.125,0.000000,8,ریاضی,اصول سیستم های کامپیوتری | اصول مدیریت | برنام...
ابوالفضل اسکندری,3.615385,4.307692,3.153846,4.076923,4.769231,3.00,0.384615,0.615385,0.000,0.00,0.000,0.307692,13,فیزیک,آز فیزیک ۱ | آز فیزیک۱ | آز ۱ | آزمایشگاه فیزی...
ابوذر غفاری,9.125000,9.875000,8.125000,6.500000,7.750000,9.25,0.200000,0.300000,0.500,0.00,0.000,0.200000,10,برق,درس مدار ۲ | مدار 1 | مدار 2 | مدار الکتریکی ۲...


In [22]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

# 1. Define Numeric Feature Vectors (The "Teaching Style")
# These features define HOW they teach (Strictness, Grades, etc.)
numeric_features = [
    'rating_1', 'rating_2', 'rating_3', 'rating_4', 'rating_5', 'rating_6',
    'is_pos', 'is_neg', 'is_neu',
    'has_project', 'has_homework', 'has_attendance'
]

# 2. Encode Categorical Data (The "Topic")
# We turn the 'department' column into numerical "Dummy Variables".
# Example: If a prof is in "Physics", they get a 1 in the "dept_Physics" column.
dept_dummies = pd.get_dummies(prof_profile['department'], prefix='dept')

# 3. Combine Style + Topic
# We join the numeric teaching style with the department topic.
# Now, professors in the same department will inherently have higher similarity!
X = prof_profile[numeric_features].join(dept_dummies).fillna(0)

# 4. Calculate Similarity Matrix
# Compute cosine similarity based on BOTH Style and Department.
similarity_matrix = cosine_similarity(X)

# Convert into a readable DataFrame for easy lookup
sim_df = pd.DataFrame(
    similarity_matrix,
    index=prof_profile.index,
    columns=prof_profile.index
)

# 5. Define the Content-Based Recommendation Function (SCENARIO 1)
def get_content_based_recommendations(prof_name, top_n=5):
    """
    Implementation of Scenario 1: Similarity Search.
    Now considers Teaching Style AND Department matching.
    """
    # Validation: Check if the professor exists
    if prof_name not in sim_df.index:
        print(f"Error: Professor '{prof_name}' not found in the database.")
        return None

    # Get similarity scores for this professor
    # Sort descending so the highest similarity (closest to 1.0) comes first
    similar_scores = sim_df[prof_name].sort_values(ascending=False)

    # Drop the professor themselves from the list
    similar_scores = similar_scores.drop(prof_name)

    # Return the top N results
    return similar_scores.head(top_n)

# 6. Verify Metadata (for later scenarios)
print("Enhanced Content-Based System (Scenario 1) Built.")
print(f"   - Features used: {len(numeric_features)} Numeric + {len(dept_dummies.columns)} Department Tags")

required_meta = ['department', 'lesson_name']
if all(col in prof_profile.columns for col in required_meta):
    print("Metadata (Department & Course) is ready for Scenario 3.")
else:
    print("WARNING: Metadata missing! Check Cell 1.")

Enhanced Content-Based System (Scenario 1) Built.
   - Features used: 12 Numeric + 15 Department Tags
Metadata (Department & Course) is ready for Scenario 3.


In [23]:
# 1. Standard Test
if len(prof_profile) > 0:
    valid_prof = prof_profile.index[0]
    prof_dept = prof_profile.loc[valid_prof, 'department']
    
    print(f"TEST 1: Standard Recommendation for '{valid_prof}'")
    print(f"Target Department: {prof_dept}")
    
    res1 = get_content_based_recommendations(valid_prof)
    
    # Display results including Department to verify the new logic
    cols_to_show = ['rating_1', 'department', 'lesson_name', 'rating_3']
    if res1 is not None:
        display(prof_profile.loc[res1.index][cols_to_show])
else:
    print("Test 1 Skipped (Data empty)")
print("-" * 50)

# 2. Error Handling
print(f"TEST 2: Invalid Name Handling")
get_content_based_recommendations("Dr. Ghost")
print("-" * 50)

# 3. Project-Based Matching
# Find a professor with projects (has_project > 0.1)
proj_profs = prof_profile[prof_profile['has_project'] > 0.1].index
if len(proj_profs) > 0:
    p_prof = proj_profs[0]
    p_dept = prof_profile.loc[p_prof, 'department']
    
    print(f"TEST 3: Project Similarity for '{p_prof}' (Dept: {p_dept})")
    print("Expectation: Recommendations should have Projects AND be in the same Dept.")
    
    res3 = get_content_based_recommendations(p_prof)
    
    if res3 is not None:
        display(prof_profile.loc[res3.index][['rating_1', 'has_project', 'department']])
else:
    print("Skipping Test 3 (No project professors found)")
print("-" * 50)

# 4. Homework Matching
hw_profs = prof_profile[prof_profile['has_homework'] > 0.1].index
if len(hw_profs) > 0:
    h_prof = hw_profs[0]
    h_dept = prof_profile.loc[h_prof, 'department']
    
    print(f"TEST 4: Homework Similarity for '{h_prof}' (Dept: {h_dept})")
    
    res4 = get_content_based_recommendations(h_prof)
    if res4 is not None:
        display(prof_profile.loc[res4.index][['rating_1', 'has_homework', 'department']])
else:
    print("Skipping Test 4 (No heavy-homework professors found)")
print("-" * 50)

# 5. Attendance Matching
att_profs = prof_profile[prof_profile['has_attendance'] > 0.1].index
if len(att_profs) > 0:
    a_prof = att_profs[0]
    a_dept = prof_profile.loc[a_prof, 'department']
    
    print(f"TEST 5: Attendance Similarity for '{a_prof}' (Dept: {a_dept})")
    
    res5 = get_content_based_recommendations(a_prof)
    if res5 is not None:
        display(prof_profile.loc[res5.index][['rating_1', 'has_attendance', 'department']])
else:
    print("Skipping Test 5 (No strict-attendance professors found)")
print("-" * 50)

print("Tests Complete.")

TEST 1: Standard Recommendation for 'ابراهیم اردشیر لاریجانی'
Target Department: ریاضی


,rating_1,department,lesson_name,rating_3
professor_name,,,,
محمدباقر قائمی,7.700000,ریاضی,مبانی آنالیز ریاضی | مبانی آنالیز-آنالیز ریاضی...,6.200000
محمد باقر قائمی,7.440000,ریاضی,دیفرانسیل | مبانی علوم ریاضی | معادلات | معادل...,6.400000
مهدی نجفی خواه,8.041667,ریاضی,توپولوژی _جبر خطی | توپولوژی عمومی-مبانی ریاضی...,7.000000
رضا احمدی,9.272727,ریاضی,آمار و احتمال | آمار و احتمال مهندسی | آمار و ...,7.772727
سید محمد کاظم حسینی پور,6.125000,ریاضی,ریاضی 1-2 | ریاضی ۱ | معادلات دیفرانسیل | نظری...,5.750000


--------------------------------------------------
TEST 2: Invalid Name Handling
Error: Professor 'Dr. Ghost' not found in the database.
--------------------------------------------------
TEST 3: Project Similarity for 'ابراهیم اردشیر لاریجانی' (Dept: ریاضی)
Expectation: Recommendations should have Projects AND be in the same Dept.


,rating_1,has_project,department
professor_name,,,
محمدباقر قائمی,7.700000,0.000000,ریاضی
محمد باقر قائمی,7.440000,0.038462,ریاضی
مهدی نجفی خواه,8.041667,0.000000,ریاضی
رضا احمدی,9.272727,0.000000,ریاضی
سید محمد کاظم حسینی پور,6.125000,0.000000,ریاضی


--------------------------------------------------
TEST 4: Homework Similarity for 'ابراهیم اردشیر لاریجانی' (Dept: ریاضی)


,rating_1,has_homework,department
professor_name,,,
محمدباقر قائمی,7.700000,0.200000,ریاضی
محمد باقر قائمی,7.440000,0.269231,ریاضی
مهدی نجفی خواه,8.041667,0.083333,ریاضی
رضا احمدی,9.272727,0.173913,ریاضی
سید محمد کاظم حسینی پور,6.125000,0.187500,ریاضی


--------------------------------------------------
TEST 5: Attendance Similarity for 'ابوالفضل اسکندری' (Dept: فیزیک)


,rating_1,has_attendance,department
professor_name,,,
حجت قاسمی,7.857143,0.125000,مکانیک
رضا ترکاشون,5.400000,0.000000,مهندسی کامپیوتر
فزرانه شیخ الاسلامی,8.000000,0.142857,فیزیک
ایت قره قانی,8.263158,0.083333,مکانیک
محمدرضا جعفرفرد,9.517241,0.379310,فیزیک


--------------------------------------------------
Tests Complete.


In [24]:
def filter_professors_by_rules(
    min_score=None, 
    min_fairness=None, 
    project_based=None,    
    heavy_homework=None,   
    strict_attendance=None,
    department=None,
    course_name=None,
    top_n=5
):
    """
    Scenario 2 Implementation:
    Filters the professor list based on strict rules (Hard Filters).
    If no filters are applied, returns the top rated professors overall.
    """
    # Start with a copy of the full data
    results = prof_profile.copy()
    
    # --- Filter 1: High Score (rating_1) ---
    if min_score is not None:
        results = results[results['rating_1'] >= min_score]
        
    # --- Filter 2: Fair Grading (rating_3) ---
    if min_fairness is not None:
        results = results[results['rating_3'] >= min_fairness]

    # --- Filter 3: Project-Based ---
    if project_based is True:
        results = results[results['has_project'] >= 0.10]
    elif project_based is False:
        results = results[results['has_project'] < 0.10]
    
    # --- Filter 4: Homework ---
    if heavy_homework is True:
        results = results[results['has_homework'] >= 0.20]
    elif heavy_homework is False:
        results = results[results['has_homework'] < 0.20]

    # --- Filter 5: Attendance ---
    if strict_attendance is True:
        results = results[results['has_attendance'] >= 0.15]
    elif strict_attendance is False:
        results = results[results['has_attendance'] < 0.15]

    # --- Filter 6: Department (NEW) ---
    # We check if 'department' is provided and is not the UI default "All"
    if department and department != "All":
        results = results[results['department'] == department]

    # --- Filter 7: Course Name (NEW) ---
    # Partial string match (case insensitive)
    if course_name:
        results = results[results['lesson_name'].str.contains(course_name, case=False, na=False)]

    # Sort by Quality (rating_1) and return the top N
    results = results.sort_values(by='rating_1', ascending=False).head(top_n)
    
    # Return relevant columns to verify (Added Dept & Lesson)
    cols_to_show = ['rating_1', 'department', 'lesson_name', 'rating_3', 'has_project', 'has_homework', 'has_attendance']
    
    # Filter columns that exist in the dataframe to prevent errors
    cols_to_show = [c for c in cols_to_show if c in results.columns]
        
    return results[cols_to_show]

print("Rule-Based Filtering Complete.")

Rule-Based Filtering Complete.


In [25]:
# 1. Test: The "Ideal" Professor
print("TEST 1: The 'Ideal' Professor (Score>8, Fair>8, Projects)")
res1 = filter_professors_by_rules(min_score=8.0, min_fairness=8.0, project_based=True)
display(res1)
print("-" * 50)

# 2. Test: The "Lazy" Student
print("TEST 2: The 'Chill' Semester (No Homework, No Attendance Checks)")
res2 = filter_professors_by_rules(heavy_homework=False, strict_attendance=False, min_score=7.0)
display(res2)
print("-" * 50)

# 3. Test: The "Hardcore" Student
print("TEST 3: The 'Hardcore' Student (Projects + Heavy Homework)")
res3 = filter_professors_by_rules(project_based=True, heavy_homework=True)
display(res3)
print("-" * 50)

# 4. Test: The "Browsing" Student
print("TEST 4: No Filters")
res4 = filter_professors_by_rules()
display(res4)
print("-" * 50)

# 5. Test: Pure Theory
print("TEST 5: Pure Theory (Project_Based = False)")
res5 = filter_professors_by_rules(project_based=False, min_score=8.0)
display(res5)
print("-" * 50)

# 6. Test: Department Search
print("TEST 6: Department Search (Example: 'برق' or 'کامپیوتر')")
# We use a word we KNOW exists in your data from previous output
res6 = filter_professors_by_rules(department="برق") 
display(res6)
print("-" * 50)

# 7. Test: Course Name Search
print("TEST 7: Course Search (Searching for 'فیزیک')")
# Searching for 'Physics' in Farsi
res7 = filter_professors_by_rules(course_name="فیزیک", min_score=0.0)
display(res7)
print("-" * 50)

# 8. Test: The "Smart Search" (Course + Chill)
print("TEST 8: Combined Search ('فیزیک' professors who are 'Chill')")
# Finds professors who teach Physics AND don't take strict attendance
res8 = filter_professors_by_rules(course_name="فیزیک", strict_attendance=False)
display(res8)
print("-" * 50)

print("Rule-Based Tests Complete."

TEST 1: The 'Ideal' Professor (Score>8, Fair>8, Projects)


,rating_1,department,lesson_name,rating_3,has_project,has_homework,has_attendance
professor_name,,,,,,,
سهیل گنجه فر,10.000000,برق,سیستم های کنترل خطی | سیستمهای کنترل خطی | کنت...,10.000000,0.800000,0.400000,0.200000
روح الدین میری,9.750000,مهندسی شیمی,ترمودینامیک 1 | ترمودینامیک ۱ | ترمودینامیک ۱ ...,10.000000,0.100000,0.200000,0.100000
ذبیح الله ذبیحی لهرمی,9.700000,فیزیک,فیزیک | فیزیک 1 | فیزیک 2 | فیزیک محاسباتی | ف...,9.400000,0.100000,0.000000,0.100000
محمد صدیقی,9.636364,مکانیک,شکل دادن فلزات | علم مواد | متالوژی در تولید,9.818182,0.428571,0.142857,0.000000
محمود مهرداد شکریه,9.625000,مکانیک,تئوری الاستیسیته-مکانیک مواد کامپوزیت | مقاومت...,9.250000,0.222222,0.666667,0.111111


--------------------------------------------------
TEST 2: The 'Chill' Semester (No Homework, No Attendance Checks)


,rating_1,department,lesson_name,rating_3,has_project,has_homework,has_attendance
professor_name,,,,,,,
محمدرضا جعفر فرد,10.000000,فیزیک,فیزیک 2 | فیزیک ۲ | فیزیک-2 | فیزیک۲,10.000000,0.000000,0.000000,0.000000
مریم زارعشاهی,10.000000,فیزیک,آزمایشگاه فیزیک ۱ | آزمایشگاه فیزیک 2 | آزمای...,10.000000,0.000000,0.000000,0.000000
نرگس طاهری,9.727273,ریاضی,رياضيات مهندسی | ریاضی مهندسی | ریاضی-مهندسی |...,9.636364,0.000000,0.178571,0.142857
ذبیح الله ذبیحی لهرمی,9.700000,فیزیک,فیزیک | فیزیک 1 | فیزیک 2 | فیزیک محاسباتی | ف...,9.400000,0.100000,0.000000,0.100000
محمد صدیقی,9.636364,مکانیک,شکل دادن فلزات | علم مواد | متالوژی در تولید,9.818182,0.428571,0.142857,0.000000


--------------------------------------------------
TEST 3: The 'Hardcore' Student (Projects + Heavy Homework)


...

--------------------------------------------------
TEST 7: Course Search (Searching for 'فیزیک')


,rating_1,department,lesson_name,rating_3,has_project,has_homework,has_attendance
professor_name,,,,,,,
مریم زارعشاهی,10.000000,فیزیک,آزمایشگاه فیزیک ۱ | آزمایشگاه فیزیک 2 | آزمای...,10.0,0.0,0.0,0.000000
محمدرضا جعفر فرد,10.000000,فیزیک,فیزیک 2 | فیزیک ۲ | فیزیک-2 | فیزیک۲,10.0,0.0,0.0,0.000000
گل اذین طاهرابادی,9.800000,فیزیک,آز فیزیک ۱ | آزمایشگاه فیزیک پایه 1 | آزمایشگا...,10.0,0.0,0.0,0.400000
ذبیح الله ذبیحی لهرمی,9.700000,فیزیک,فیزیک | فیزیک 1 | فیزیک 2 | فیزیک محاسباتی | ف...,9.4,0.1,0.0,0.100000
حسین سرپولکی,9.666667,مهندسی مواد,خواص فيزيكي مواد ١ | خواص فیزیکی 1 - خواص فیزی...,10.0,0.0,0.0,0.166667


--------------------------------------------------
TEST 8: Combined Search ('فیزیک' professors who are 'Chill')


,rating_1,department,lesson_name,rating_3,has_project,has_homework,has_attendance
professor_name,,,,,,,
محمدرضا جعفر فرد,10.000,فیزیک,فیزیک 2 | فیزیک ۲ | فیزیک-2 | فیزیک۲,10.000,0.0,0.0,0.000
مریم زارعشاهی,10.000,فیزیک,آزمایشگاه فیزیک ۱ | آزمایشگاه فیزیک 2 | آزمای...,10.000,0.0,0.0,0.000
ذبیح الله ذبیحی لهرمی,9.700,فیزیک,فیزیک | فیزیک 1 | فیزیک 2 | فیزیک محاسباتی | ف...,9.400,0.1,0.0,0.100
حمید رضا جهانگیری,9.625,مهندسی شیمی,آشنایی با مهندسی نفت | آشنایی با مهندسی نفت - ...,7.625,0.0,0.0,0.125
حمیدرضا مازندرانی,9.600,فیزیک,آز فیزیک ۲ | آزمایشگاه فیزیک 2 | آزمایشگاه فیز...,8.400,0.0,0.0,0.000


--------------------------------------------------
Rule-Based Tests Complete.


In [26]:
def get_hybrid_recommendation(
    target_prof=None,      # Option 1: Find Similar to this person
    min_score=None,        # Option 2: Filter by Score
    project_based=None,    # Tri-State: True, False, None
    heavy_homework=None,   # Tri-State
    strict_attendance=None,# Tri-State
    department=None,       # <--- NEW: Filter by Dept
    course_name=None,      # <--- NEW: Filter by Course
    top_n=5
):
    """
    Unified Recommender:
    - If 'target_prof' is given: Finds similar professors who meet the criteria.
    - If 'target_prof' is None: Finds top-rated professors who meet the criteria.
    - Applies all filters including Dept & Course.
    """
    
    # --- STEP 1: Determine Candidate Pool ---
    if target_prof:
        # PATH A: Similarity Search
        if target_prof not in sim_df.index:
            print(f"Error: Professor '{target_prof}' not found.")
            return None
        
        # Get top 50 similar professors (sorted by similarity)
        # Note: sim_df now includes Department similarity from Cell 2!
        similar_candidates_index = sim_df[target_prof].sort_values(ascending=False).drop(target_prof).head(50).index
        candidates = prof_profile.loc[similar_candidates_index].copy()
    else:
        # PATH B: Global Search (No Name)
        # Start with EVERYONE
        candidates = prof_profile.copy()
    
    # --- STEP 2: Rule-Based Filtering ---
    
    # 1. Score Filter
    if min_score is not None:
        candidates = candidates[candidates['rating_1'] >= min_score]
        
    # 2. Project Filter (Tri-State)
    if project_based is True:
        candidates = candidates[candidates['has_project'] >= 0.10]
    elif project_based is False:
        candidates = candidates[candidates['has_project'] < 0.10]

    # 3. Homework Filter (Tri-State)
    if heavy_homework is True:
        candidates = candidates[candidates['has_homework'] >= 0.20]
    elif heavy_homework is False:
        candidates = candidates[candidates['has_homework'] < 0.20]

    # 4. Attendance Filter (Tri-State)
    if strict_attendance is True:
        candidates = candidates[candidates['has_attendance'] >= 0.15]
    elif strict_attendance is False:
        candidates = candidates[candidates['has_attendance'] < 0.15]

    # 5. Department Filter (NEW)
    if department and department != "All":
        candidates = candidates[candidates['department'] == department]

    # 6. Course Name Filter (NEW)
    if course_name:
        # Partial match on 'lesson_name' (e.g., "Physics" matches "Physics I")
        candidates = candidates[candidates['lesson_name'].str.contains(course_name, case=False, na=False)]

    # --- STEP 3: Sorting & Return ---
    
    if target_prof is None:
        # If we didn't search by similarity, we must sort by Quality (Rating)
        candidates = candidates.sort_values(by='rating_1', ascending=False)
    
    # (If target_prof WAS provided, 'candidates' is already sorted by similarity from Step 1)

    # Added 'department' and 'lesson_name' to columns so you can verify the results
    cols_to_show = ['rating_1', 'department', 'lesson_name', 'has_project', 'has_homework', 'has_attendance']
    
    # Safety check to prevent errors if columns are missing
    cols_to_show = [c for c in cols_to_show if c in candidates.columns]
    
    return candidates[cols_to_show].head(top_n)

print("Hybrid System Finalized.")

Hybrid System Finalized.


In [27]:
# Choose a target that exists in your data
target = prof_profile.index[0] 
# (Or manually set: target = 'اسماء سلیمانی' if she exists)

# --- PART 1: WITH NAME (Similarity + Filter) ---
print("--- PART 1: WITH NAME (Similarity Mode) ---")

# Test 1.1: "Like Target, but STRICTLY Theory (No Projects)"
print(f"TEST 1.1: Like '{target}' + Project_Based=False")
res1 = get_hybrid_recommendation(target_prof=target, project_based=False)
display(res1)
print("-" * 30)

# Test 1.2: "Like Target, but STRICTLY Heavy Homework"
print(f"TEST 1.2: Like '{target}' + Heavy_Homework=True")
res2 = get_hybrid_recommendation(target_prof=target, heavy_homework=True)
display(res2)
print("-" * 30)

# Test 1.3: "Like Target, but CHILL Attendance"
print(f"TEST 1.3: Like '{target}' + Strict_Attendance=False")
res3 = get_hybrid_recommendation(target_prof=target, strict_attendance=False)
display(res3)
print("-" * 50)


# --- PART 2: NO NAME (Discovery Mode) ---
print("--- PART 2: NO NAME (Discovery Mode) ---")

# Test 2.1: "Just find me Project Professors (Score > 9)"
print(f"TEST 2.1: Any Prof + Project_Based=True + Score > 9")
res4 = get_hybrid_recommendation(target_prof=None, min_score=9.0, project_based=True)
display(res4)
print("-" * 30)

# Test 2.2: "Just find me Light Homework Professors"
print(f"TEST 2.2: Any Prof + Heavy_Homework=False")
res5 = get_hybrid_recommendation(target_prof=None, heavy_homework=False)
display(res5)
print("-" * 30)

# Test 2.3: "The 'Strict' Search (Strict Attendance + Heavy Homework)"
print(f"TEST 2.3: Any Prof + Strict_Attendance=True + Heavy_Homework=True")
res6 = get_hybrid_recommendation(target_prof=None, strict_attendance=True, heavy_homework=True)
display(res6)
print("-" * 50)


# --- PART 3: NEW SEARCH FEATURES (Dept & Course) ---
print("--- PART 3: SEARCH FILTERS (Dept & Course) ---")

# Test 3.1: Course Search (e.g., 'Physics')
print("TEST 3.1: Search for 'فیزیک' (Physics) Professors")
res7 = get_hybrid_recommendation(course_name="فیزیک")
display(res7)
print("-" * 30)

# Test 3.2: Dept Search + Chill Mode
# Find a 'Computer' prof who doesn't care about attendance
# (Replace 'کامپیوتر' with a department name from your data like 'برق' if needed)
print("TEST 3.2: Dept='برق' + Strict_Attendance=False")
res8 = get_hybrid_recommendation(department="برق", strict_attendance=False)
display(res8)
print("-" * 30)

# Test 3.3: The "unicorn" search (Similarity + Course)
# "Find someone like [Target] who teaches [Course]"
print(f"TEST 3.3: Like '{target}' + Course='ریاضی' (Math)")
res9 = get_hybrid_recommendation(target_prof=target, course_name="ریاضی")
display(res9)
print("-" * 50)

print("Comprehensive Tests Complete.")

--- PART 1: WITH NAME (Similarity Mode) ---
TEST 1.1: Like 'ابراهیم اردشیر لاریجانی' + Project_Based=False


,rating_1,department,lesson_name,has_project,has_homework,has_attendance
professor_name,,,,,,
محمدباقر قائمی,7.700000,ریاضی,مبانی آنالیز ریاضی | مبانی آنالیز-آنالیز ریاضی...,0.000000,0.200000,0.400000
محمد باقر قائمی,7.440000,ریاضی,دیفرانسیل | مبانی علوم ریاضی | معادلات | معادل...,0.038462,0.269231,0.153846
مهدی نجفی خواه,8.041667,ریاضی,توپولوژی _جبر خطی | توپولوژی عمومی-مبانی ریاضی...,0.000000,0.083333,0.208333
رضا احمدی,9.272727,ریاضی,آمار و احتمال | آمار و احتمال مهندسی | آمار و ...,0.000000,0.173913,0.173913
سید محمد کاظم حسینی پور,6.125000,ریاضی,ریاضی 1-2 | ریاضی ۱ | معادلات دیفرانسیل | نظری...,0.000000,0.187500,0.250000


------------------------------
TEST 1.2: Like 'ابراهیم اردشیر لاریجانی' + Heavy_Homework=True


,rating_1,department,lesson_name,has_project,has_homework,has_attendance
professor_name,,,,,,
محمدباقر قائمی,7.700000,ریاضی,مبانی آنالیز ریاضی | مبانی آنالیز-آنالیز ریاضی...,0.000000,0.200000,0.400000
محمد باقر قائمی,7.440000,ریاضی,دیفرانسیل | مبانی علوم ریاضی | معادلات | معادل...,0.038462,0.269231,0.153846
هانیه فلاح اسلاملو,7.625000,ریاضی,ریاضی ۱ | معادلات دیفرانسیل,0.000000,0.375000,0.250000
رضا انتظاری ملکی,8.166667,مهندسی کامپیوتر,برنامه نویسی کامپیوتر | مباحث ویژه نرم افزار ۱...,0.416667,0.583333,0.166667
فروغ ثنائی,8.625000,ریاضی,ریاضی عمومی ۱ | ریاضی عمومی۱ | محاسبات عددی,0.375000,0.250000,0.250000


------------------------------
TEST 1.3: Like 'ابراهیم اردشیر لاریجانی' + Strict_Attendance=False


...

------------------------------
TEST 3.2: Dept='برق' + Strict_Attendance=False


,rating_1,department,lesson_name,has_project,has_homework,has_attendance
professor_name,,,,,,
محمود جواد جنتی,9.800000,برق,ریاضیات رمزنگاری | سیگنال سیستم | سیگنال و سیس...,0.000000,0.200000,0.000000
حمیدرضا جلیلیان خالقی,8.800000,برق,آز مدار | آز مدار ۱ | آزمایشگاه مدار منطقی | آ...,0.000000,0.000000,0.000000
سید ادیب ابریشمی فر,8.555556,برق,الکترونیک ۱ | الکترونیک ۳ | الکترونیک1 - الکتر...,0.222222,0.111111,0.000000
داوود عرب خابوری,7.882353,برق,مدار الکتریکی 1 | مدار الکتریکی ۱ | مدار الکتر...,0.000000,0.235294,0.117647
هادی شهریار شاه حسینی,6.250000,برق,اجزا کامپیوتر | اجزای کامپیوتر | مدار واسط | م...,0.500000,0.250000,0.000000


------------------------------
TEST 3.3: Like 'ابراهیم اردشیر لاریجانی' + Course='ریاضی' (Math)


,rating_1,department,lesson_name,has_project,has_homework,has_attendance
professor_name,,,,,,
محمدباقر قائمی,7.700000,ریاضی,مبانی آنالیز ریاضی | مبانی آنالیز-آنالیز ریاضی...,0.000000,0.200000,0.400000
محمد باقر قائمی,7.440000,ریاضی,دیفرانسیل | مبانی علوم ریاضی | معادلات | معادل...,0.038462,0.269231,0.153846
مهدی نجفی خواه,8.041667,ریاضی,توپولوژی _جبر خطی | توپولوژی عمومی-مبانی ریاضی...,0.000000,0.083333,0.208333
رضا احمدی,9.272727,ریاضی,آمار و احتمال | آمار و احتمال مهندسی | آمار و ...,0.000000,0.173913,0.173913
سید محمد کاظم حسینی پور,6.125000,ریاضی,ریاضی 1-2 | ریاضی ۱ | معادلات دیفرانسیل | نظری...,0.000000,0.187500,0.250000


--------------------------------------------------
Comprehensive Tests Complete.
